In [19]:
from torch_uncertainty import cli_main, init_args
from torch_uncertainty.baselines.classification import ResNet
from torch_uncertainty.optimization_procedures import optim_cifar10_resnet18
from torch_uncertainty.datamodules import CIFAR10DataModule
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.nn import CrossEntropyLoss
import torch

import os
import pandas as pd
from pathlib import Path
from cli_test_helpers import ArgvContext

In [20]:
torch.cuda.is_available()

True

### 1 - Models

In [21]:
baseline = ResNet(num_classes=10,
                loss=CrossEntropyLoss,
                optimization_procedure=optim_cifar10_resnet18,
                version="std",
                in_channels=3, 
                arch=18,
                mixup=False,
                regmixup=False).cuda()

mixup = ResNet(num_classes=10,
                loss=CrossEntropyLoss,
                optimization_procedure=optim_cifar10_resnet18,
                version="std",
                in_channels=3, 
                arch=18, 
                mixup=True,
                mixup_alpha=0.2).cuda()

regmixup = ResNet(num_classes=10,
                loss=CrossEntropyLoss,
                optimization_procedure=optim_cifar10_resnet18,
                version="std",
                in_channels=3,
                arch=18,
                reg_mixup=True,
                mixup_alpha=15).cuda()

### 2 - Data

In [22]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 3. Training

In [23]:
root = Path(os.path.abspath(""))

# We mock the arguments for the trainer
with ArgvContext(
    "file.py",
    "--max_epochs",
    "20",
    "--enable_progress_bar",
    "False",
    "--num_estimators",
    "8"
):
    args = init_args(network=ResNet, datamodule=CIFAR10DataModule)

args.corruption_severity = 40
net_name = "logs/reset18-cifar10"

# datamodule
args.root = str(root / "data")
dm = CIFAR10DataModule(**vars(args))


In [24]:
#Trainer(accelerator="gpu", devices=1)
results_baseline = cli_main(baseline, dm, root, net_name, args=args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params
------------------------------------------------------
0 | model            | _ResNet          | 11.2 M
1 | format_batch_fn  | Identity         | 0     
2 | val_cls_metrics  | MetricCollection | 0     
3 | test_cls_metrics | MetricCollection | 0     
4 | test_entropy_id  | Entropy          | 0     
------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_29\checkpoints\epoch=15-step=6256.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_29\checkpoints\epoch=15-step=6256.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      cls_test/acc          0.7494999766349792
     cls_test/brier         0.35027873516082764
      cls_test/ece          0.03469502925872803
    cls_test/entropy        0.6254900693893433
      cls_test/nll           0.741416871547699
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [25]:
results_mixup = cli_main(mixup, dm, root, net_name, args=args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params
------------------------------------------------------
0 | model            | _ResNet          | 11.2 M
1 | format_batch_fn  | Identity         | 0     
2 | val_cls_metrics  | MetricCollection | 0     
3 | test_cls_metrics | MetricCollection | 0     
4 | test_entropy_id  | Entropy          | 0     
------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_30\checkpoints\epoch=18-step=7429.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_30\checkpoints\epoch=18-step=7429.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      cls_test/acc          0.7577000260353088
     cls_test/brier         0.3386794626712799
      cls_test/ece         0.027453485876321793
    cls_test/entropy        0.6478248834609985
      cls_test/nll           0.704408586025238
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [26]:
results_regmixup = cli_main(regmixup, dm, root, net_name, args=args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params
------------------------------------------------------
0 | model            | _ResNet          | 11.2 M
1 | format_batch_fn  | Identity         | 0     
2 | val_cls_metrics  | MetricCollection | 0     
3 | test_cls_metrics | MetricCollection | 0     
4 | test_entropy_id  | Entropy          | 0     
------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_31\checkpoints\epoch=19-step=7820.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_31\checkpoints\epoch=19-step=7820.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      cls_test/acc          0.7498999834060669
     cls_test/brier         0.35079556703567505
      cls_test/ece          0.04187669977545738
    cls_test/entropy        0.6246733665466309
      cls_test/nll          0.7313908934593201
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [27]:
results = pd.DataFrame([results_baseline[0], results_mixup[0], results_regmixup[0]])
results.index = ["baseline", "mixup", "regmixup"]
results.columns = ["entropy", "accuracy", "brier", "ece", "nll"]
results

,entropy,accuracy,brier,ece,nll
baseline,0.625490,0.7495,0.350279,0.034695,0.741417
mixup,0.647825,0.7577,0.338679,0.027453,0.704409
regmixup,0.624673,0.7499,0.350796,0.041877,0.731391
